# Automated ML


In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

In [2]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'puru-auto-experiment'

experiment=Experiment(ws, experiment_name)

In [4]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-132677
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-132677


In [5]:
# create or attach an existing compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# set name of CPU cluster
cpu_cluster_name = "puru-compute-new"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, using it.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# wait for completion
cpu_cluster.wait_for_completion(show_output=True)

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
I am using external data source from Kaggle. I have already downloaded the data from Kaggle and uploaded in my github project, so that we can give the data source as url. 
The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. The target (label) is "Outcome" column with values of 1 means has diabetes and 0 means no diabetes. We will be using all independent varialbles from the datasets like: preganicies the patient has had, their BMI, insulin level, age and so on.

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# NOTE: update the key to match the dataset name
found = False
key = "diabetes-classification"
description_text = "Diabetes datasets for capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url = 'https://raw.githubusercontent.com/purunep/Capstoneproject/main/project/data/diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
# prepare dataset for model training
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Clean dataset using the clean_data function
x, y = clean_data(dataset)

# split dataset into train and test sets
(x_train, x_test, y_train, y_test) = train_test_split(x, y, test_size= 0.2, random_state = 0)
label = 'Outcome'

# merge the output x and y dataframes into a single table for AutoML experiment
train_data_df = pd.concat([x_train, y_train], axis=1)

train_data_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
603,7,150,78,29,126,35.2,0.692,54,1
118,4,97,60,23,0,28.2,0.443,22,0
247,0,165,90,33,680,52.3,0.427,23,0
157,1,109,56,21,135,25.2,0.833,23,0
468,8,120,0,0,0,30.0,0.183,38,1


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory
# save training data in tabular format to allow for remote run
if not os.path.isdir('data'):  # create data folder if it does not exist
    os.mkdir('data')
    
if not os.path.exists('project_folder'):  # create project folder if it does not exist
    os.makedirs('project_folder')

# Save the train data to a csv file to be uploaded to the datastore
pd.DataFrame(train_data_df).to_csv("data/train_data.csv", index=False)

# Upload the training data as a tabular dataset for access during training on remote compute
# upload to data store
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='automlclassifier', overwrite=True, show_progress=True)

 # access datastore during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('automlclassifier/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

I set enabled early stopping to True, to enable early termination if the score is not improving in the short term. Also set the iteration_timeout_minutes = 5, to set the maximum time in minutes that each iteration can run for before it terminates.Set max_concurrent_iterations = 4, to specify the maximum number of iterations that would be executed in parallel. Set max_cores_per_iteration = -1, to specify the maximum number of threads to use for a given training iteration. -1 means to use all the possible cores per iteration per child-run and set featurization = auto, to specify wherether featurization should be done automically or not, auto is ued to do it automatically

In [9]:
project_folder = './puruautoml'
automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,    
    "featurization": 'auto'
}

# define automl autconfig parameters
automl_config = AutoMLConfig(
    experiment_timeout_minutes= 30,
    task= 'classification',
    primary_metric='accuracy',
    enable_onnx_compatible_models=True,
    compute_target=cpu_cluster, # included to allow for remote compute
    training_data= train_data,
    label_column_name= label,
    path = project_folder,
    n_cross_validations=3,
    debug_log = "automl_errors.log",    
    **automl_settings)

In [10]:
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

# wait for completion
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

********************************

{'runId': 'AutoML_ceb5a17f-97e5-4896-83ac-c5e4c567d2c1',
 'target': 'puru-compute-new',
 'status': 'Completed',
 'startTimeUtc': '2020-12-31T18:13:26.043787Z',
 'endTimeUtc': '2020-12-31T18:36:54.312723Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'puru-compute-new',
  'AMLSettingsJsonString': '{"path":null,"name":"puru-auto-experiment","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-132677","workspace_name":"quick-starts-ws-132677","region":"southcentralus","compute_target":"puru-compute-new","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null

## Best Model



In [12]:
# Retrieve the best automl run model
best_automl_run, fitted_automl_model = remote_run.get_output()
print(best_automl_run)

# get best model and display properties
model_name = best_automl_run.properties['model_name']
print("Best Model:", model_name)

# display all the properties of the best model
#best_automl_run #.properties
best_automl_run.get_properties()

Run(Experiment: puru-auto-experiment,
Id: AutoML_ceb5a17f-97e5-4896-83ac-c5e4c567d2c1_36,
Type: azureml.scriptrun,
Status: Completed)
Best Model: AutoMLceb5a17f936


{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'puru-auto-experiment\',\'compute_target\':\'puru-compute-new\',\'subscription_id\':\'6971f5ac-8af1-446e-8034-05acea24681f\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_ceb5a17f-97e5-4896-83ac-c5e4c567d2c1_36","experiment_name":"puru-auto-experiment","workspace_name":"quick-starts-ws-132677","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group_name":"aml-quickstarts-132677"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '36',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azurem

In [13]:
# Save the best model
os.makedirs('outputs', exist_ok=True)
best_automl_run.download_file('outputs/model.pkl', 'outputs/automlmodel.pkl')
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_automl_run.download_file('automl_driver.py', 'outputs/automl_driver.py')

In [15]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ="outputs/automlmodel.pkl",
                        model_name = "automl_puru")

Registering model automl_puru


In [17]:

print(model.name)

automl_puru


## Model Deployment

In [18]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

In [19]:
%%writefile autoscore.py
import json
import numpy as np
import os
import pickle
import joblib
from azureml.core.model import Model
import pandas as pd

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automlmodel.pkl')
    model = joblib.load(model_path)
    

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        mypredict = model.predict(data)
        return mypredict.tolist()
    except Exception as ex:
        error = str(ex)
        return error

Overwriting autoscore.py


In [20]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


#env = Environment.from_conda_specification(name='puruenv',file_path = 'env.yml')
env = Environment.get(workspace=ws, name="AzureML-AutoML")
inference_config = InferenceConfig(entry_script='./autoscore.py',
                                    environment=env)

In [21]:
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights = True, auth_enabled=True)
service = Model.deploy(ws, "puruautoservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [22]:
servicelogs = service.get_logs()
print(servicelogs)

2020-12-31T18:50:25,498644502+00:00 - gunicorn/run 
2020-12-31T18:50:25,498873918+00:00 - rsyslog/run 
2020-12-31T18:50:25,498873918+00:00 - iot-server/run 
2020-12-31T18:50:25,536924303+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [23]:

print("scoring URI: " + service.scoring_uri)

print("Swagger URI: " + service.swagger_uri)

print("Authetication Key: " + service.get_keys()[0])

scoring URI: http://7949d0dc-6cf5-49c5-ba5d-80823b6ea794.southcentralus.azurecontainer.io/score
Swagger URI: http://7949d0dc-6cf5-49c5-ba5d-80823b6ea794.southcentralus.azurecontainer.io/swagger.json
Authetication Key: kdrlEKVBpaU7nI2Ocdg6ZSWYIdfDvhaX


In [26]:
from train import clean_data
[x_data, y_data] = clean_data(dataset)
print(x_data[0:2].values.tolist())

[[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0], [1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0]]


In [28]:
import json

input_payload = json.dumps({
    'data': x_data[0:2].to_dict(orient='records'),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

[1, 0]


In [24]:
import json


payload = json.dumps({
    'data':
           [
               {
                   'Pregnancies': 6,
                   'Glucose': 148,
                   'BloodPressure': 72,
                   'SkinThickness': 35,
                   'Insulin': 0,
                   'BMI': 33.6,
                   'DiabetesPedigreeFunction': 0.627,
                   'Age': 50

               },
               {
                   'Pregnancies': 1,
                   'Glucose': 85,
                   'BloodPressure': 66,
                   'SkinThickness': 29,
                   'Insulin': 0,
                   'BMI': 26.6,
                   'DiabetesPedigreeFunction': 0.351,
                   'Age': 31,  
               }
           ],
    'method': 'predict_proba'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(payload)
print(output)


[1, 0]


In [25]:
import requests
import json
scoringuri = service.scoring_uri
key = service.get_keys()[0]
data= { "data":
       [
           {
               'Pregnancies': 6,
               'Glucose': 148,
               'BloodPressure': 72,
               'SkinThickness': 35,
               'Insulin': 0,
               'BMI': 33.6,
               'DiabetesPedigreeFunction': 0.627,
               'Age': 50
               
           },
           {
               'Pregnancies': 1,
               'Glucose': 85,
               'BloodPressure': 66,
               'SkinThickness': 29,
               'Insulin': 0,
               'BMI': 26.6,
               'DiabetesPedigreeFunction': 0.351,
               'Age': 31
           }
       ]
    }
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)
print(response.text)

[1, 0]


In [ ]:
service.delete()

In [ ]:
cpu_cluster.delete()